In [1]:
from pathlib import Path
from rich import print as printr
from omegaconf import OmegaConf
import pandas as pd
from dacbench.logger import Logger, log2dataframe, flatten_log_entry, list_to_tuple
import matplotlib.pyplot as plt
from dacbench.plotting import plot_performance, plot_performance_per_instance, plot_state
from multiprocessing import Pool
import os
import time
from typing import List
import json
from functools import reduce

cfg_fn = ".hydra/config.yaml"
perf_fn = "PerformanceTrackingWrapper.jsonl"
state_fn = "StateTrackingWrapper.jsonl"
reward_fn = "RewardTrackingWrapper.jsonl"
action_fn = "ActionFrequencyWrapper.jsonl"


def log2dataframe(
    logs: List[dict], wide: bool = False, drop_columns: List[str] = ["time"]
) -> pd.DataFrame:
    """
    Converts a list of log entries to a pandas dataframe.

    Usually used in combination with load_dataframe.

    Parameters
    ----------
    logs: List
        List of log entries
    wide: bool
        wide=False (default) produces a dataframe with columns (episode, step, time, name, value)
        wide=True returns a dataframe (episode, step, time, name_1, name_2, ...) if the variable name_n has not been logged
        at (episode, step, time) name_n is NaN.
    drop_columns: List[str]
        List of column names to be dropped (before reshaping the long dataframe) mostly used in combination
        with wide=True to reduce NaN values

    Returns
    -------
    dataframe

    """
    # flat_logs = map(flatten_log_entry, logs)
    # print(flat_logs)
    # exit
    # rows = reduce(lambda l1, l2: l1 + l2, flat_logs)
    rows = logs

    dataframe = pd.DataFrame(rows)
    dataframe.time = pd.to_datetime(dataframe.time)

    if drop_columns is not None:
        dataframe = dataframe.drop(columns=drop_columns)

    dataframe = dataframe.infer_objects()
    list_column_candidates = dataframe.dtypes == object

    for i, candidate in enumerate(list_column_candidates):
        if candidate:
            dataframe.iloc[:, i] = dataframe.iloc[:, i].apply(
                lambda x: list_to_tuple(x) if isinstance(x, list) else x
            )

    if wide:
        primary_index_columns = ["episode", "step"]
        field_id_column = "name"
        additional_columns = list(
            set(dataframe.columns)
            - set(primary_index_columns + ["time", "value", field_id_column])
        )
        index_columns = primary_index_columns + additional_columns + [field_id_column]
        dataframe = dataframe.set_index(index_columns)
        dataframe = dataframe.unstack()
        dataframe.reset_index(inplace=True)
        dataframe.columns = [a if b == "" else b for a, b in dataframe.columns]

    return dataframe.infer_objects()


def load_logs(log_file: Path) -> list[dict]:
    """
    Loads the logs from a jsonl written by any logger.

    The result is the list of dicts in the format:
    {
        'instance': 0,
        'episode': 0,
        'step': 1,
        'example_log_val':  {
            'values': [val1, val2, ... valn],
            'times: [time1, time2, ..., timen],
        }
        ...
    }

    Parameters
    ----------
    log_file: pathlib.Path
        The path to the log file

    Returns
    -------
    [Dict, ...]

    """
    def f(a, b):
        return a+b
    logs = []
    with open(log_file, "r") as log_file:
        for line in log_file:
        # logs = list(map(json.loads, log_file))
            s = json.loads(line)
            s = flatten_log_entry(s)[0]
            # s = f(s)
            logs.append(s)

    return logs


def get_eval_df(eval_dir: Path) -> pd.DataFrame:
    # Get config
    cfg_filename = eval_dir / "../../.." / cfg_fn

    cfg = OmegaConf.load(cfg_filename)

    # Recover the correct test set path bc it gets overwritten
    cfg.benchmark.config.test_set_path = str(
        Path(cfg.benchmark.config.test_set_path).parent / (str(eval_dir.stem) + ".csv")
    )

    cfg_dict = OmegaConf.to_container(cfg=cfg, resolve=True)

    cfg_dict_flat = pd.json_normalize(data=cfg_dict, sep=".")

    cfg_small = {
        "benchmark_id": cfg.benchmark_id,
        "instance_set_id": cfg.instance_set_id,
        "test_set_id": Path(cfg.benchmark.config.test_set_path).name,
    }

    # Read performance data
    # s = time.time()
    # logs = load_logs(eval_dir / perf_fn)
    # print(f"Load logs perf {time.time() - s:.4f}")
    # perf_df = log2dataframe(logs, wide=True)
    # s = time.time()
    # print(f"To DF {time.time() - s:.4f}")

    # Read state data
    # logs = load_logs(eval_dir / state_fn)
    # state_df = log2dataframe(logs, wide=True)

    # Read reward data
    s = time.time()
    logs = load_logs(eval_dir / reward_fn)
    print(f"Load logs reward {time.time() - s:.4f}")
    s = time.time()
    reward_df = log2dataframe(logs, wide=True)
    print(f"To DF {time.time() - s:.4f}")

    # Read action data
    s = time.time()
    logs = load_logs(eval_dir / action_fn)
    print(f"Load logs action {time.time() - s:.4f}")
    s = time.time()
    action_df = log2dataframe(logs, wide=True)
    print(f"To DF {time.time() - s:.4f}")

    index_columns = ["episode", "step", "seed", "instance"]

    # df = perf_df
    # df = perf_df.merge(state_df)
    # df = perf_df.merge(reward_df)
    df = reward_df.merge(action_df)

    for k, v in cfg_small.items():
        df[k] = v

    print("Done", eval_dir)

    return df


def load_traineval_trajectories(path: str, train_set_id: str) -> pd.DataFrame:
    path = Path(path)
    eval_dirs = list(path.glob(f"**/eval/{train_set_id}*"))
    eval_dirs.sort()
    printr(eval_dirs)
    common_path = os.path.commonpath(eval_dirs)
    df_fn = Path("data") / common_path / "eval.csv"
    df_fn.parent.mkdir(parents=True, exist_ok=True)

    # with Pool(processes=1) as pool:
    #     dfs = pool.map(get_eval_df, eval_dirs)
    dfs = [get_eval_df(d) for d in eval_dirs]
    df = pd.concat(dfs).reset_index(drop=True)
    df.to_csv(df_fn, index=False)
    printr("Saved to", df_fn)
    return df


if __name__ == "__main__":
    path, train_set_id = Path("../runs/CMA-ES/default/ppo_sb3/full"), "cma_train"
    path, train_set_id = Path("../runs/Sigmoid/2D3M_train/ppo/full"), "sigmoid_2D3M_train"
    path, train_set_id = Path("../runs/CMA-ES/seplow_train/ppo_sb3/full"), "train"
    df = load_traineval_trajectories(path=path, train_set_id=train_set_id)


/scratch/hpc-prf-intexml/cbenjamins/envs/instancedac/lib/python3.11/site-packages/glfw/__init__.py:916: GLFWError: (65544) b'X11: The DISPLAY environment variable is missing'
  warnings.warn(message, GLFWError)


pygame 2.5.2 (SDL 2.28.2, Python 3.11.6)
Hello from the pygame community. https://www.pygame.org/contribute.html
DACBench Gym registration failed - make sure you have all dependencies installed and their instance sets in the right path!


/scratch/hpc-prf-intexml/cbenjamins/repos/instance-dac/DACBench/dacbench/envs/__init__.py:46: UserWarning: SGD Benchmark not installed. If you want to use this benchmark, please follow the installation guide.
  warnings.warn(
/scratch/hpc-prf-intexml/cbenjamins/repos/instance-dac/DACBench/dacbench/benchmarks/__init__.py:37: UserWarning: SGD Benchmark not installed. If you want to use this benchmark, please follow the installation guide.
  warnings.warn(


[
    PosixPath('../runs/CMA-ES/seplow_train/ppo_sb3/full/1/logs/eval/train'),
    PosixPath('../runs/CMA-ES/seplow_train/ppo_sb3/full/10/logs/eval/train'),
    PosixPath('../runs/CMA-ES/seplow_train/ppo_sb3/full/2/logs/eval/train'),
    PosixPath('../runs/CMA-ES/seplow_train/ppo_sb3/full/3/logs/eval/train'),
    PosixPath('../runs/CMA-ES/seplow_train/ppo_sb3/full/4/logs/eval/train'),
    PosixPath('../runs/CMA-ES/seplow_train/ppo_sb3/full/5/logs/eval/train'),
    PosixPath('../runs/CMA-ES/seplow_train/ppo_sb3/full/6/logs/eval/train'),
    PosixPath('../runs/CMA-ES/seplow_train/ppo_sb3/full/7/logs/eval/train'),
    PosixPath('../runs/CMA-ES/seplow_train/ppo_sb3/full/8/logs/eval/train'),
    PosixPath('../runs/CMA-ES/seplow_train/ppo_sb3/full/9/logs/eval/train')
]

Load logs reward 17.7912
To DF 196.1815
Load logs action 18.3828
To DF 199.9915
Done ../runs/CMA-ES/seplow_train/ppo_sb3/full/1/logs/eval/train
Load logs reward 17.3894
To DF 198.5418
Load logs action 18.5982
To DF 202.4081
Done ../runs/CMA-ES/seplow_train/ppo_sb3/full/10/logs/eval/train
Load logs reward 17.7828
To DF 208.6481
Load logs action 19.2454
To DF 206.8474
Done ../runs/CMA-ES/seplow_train/ppo_sb3/full/2/logs/eval/train
Load logs reward 19.0164
To DF 210.0202
Load logs action 19.7056
To DF 213.2569
Done ../runs/CMA-ES/seplow_train/ppo_sb3/full/3/logs/eval/train
Load logs reward 18.9348
To DF 217.0557
Load logs action 19.6505
To DF 210.9765
Done ../runs/CMA-ES/seplow_train/ppo_sb3/full/4/logs/eval/train
Load logs reward 18.8688
To DF 217.7271
Load logs action 19.3949
